## Installing default packages

In [1]:
using Pkg;

# packages to install, first time can take a while downloading, please be patient ...
packages=["HTTP","JSON","DataFrames","Plots","StatPlots","PlotThemes","GR","PyPlot","PyCall","LaTeXStrings","Plotly","PlotlyJS"]

for package in packages
    if get(Pkg.installed(),package,-1) == -1
        println(" getting package : ", package)
        Pkg.add(package)
    end
end

using Pkg;
Pkg.add(PackageSpec(url="https://github.com/DennisRutjes/Binance.jl",rev="master"));

ENV["BINANCE_APIKEY"] = "N.A.";
ENV["BINANCE_SECRET"] = "N.A.";


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/DennisRutjes/Binance.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


## Creating generic functions for Binance API data to DataFrame conversion

In [2]:
function getBinanceKlineDataframe(symbol, startDateTime, endDateTime ; interval="1m")
    klines = Binance.getKlines(symbol; startDateTime=startDateTime, endDateTime=endDateTime, interval= interval)
    result = hcat(map(z -> map(x -> typeof(x) == String ? parse(Float64, x) : x, z), klines)...)';

    #println(size(result))
    if size(result,2) == 0
        return nothing
    end

    symbolColumnData = map(x -> symbol, collect(1:size(result, 1)));
    df = DataFrame([symbolColumnData, Dates.unix2datetime.(result[:,1]/1000) ,result[:,2],result[:,3],result[:,4],result[:,5],result[:,6],result[:,8],Dates.unix2datetime.(result[:,7] / 1000),result[:,9],result[:,10],result[:,11]], [:symbol,:startDate,:open,:high,:low,:close,:volume,:quoteAVolume, :endDate, :trades, :tbBaseAVolume,:tbQuoteAVolume]);
end

# getFloatValueArray
getFloatValueArray(withKey, withDictArr) = map(dict -> parse(Float32, dict[withKey]) ,withDictArr);
getStringValueArray(withKey, withDictArr) = map(dict -> convert(String, dict[withKey]) ,withDictArr);

## Retrieve Market Symbols ending with BTC

In [3]:
using DataFrames,Statistics, Dates,Binance;

# get 24H tickerdata of all assets
hr24 = Binance.get24HR();
# previous post we determined that BTC currency was the most obvious currency to tade in, most avalable symbols, second highest USDT $ volume.
hr24BTC = Binance.filterOnRegex("BTC\$", hr24)

# all symbols ending with BTC
symbols = sort!(getStringValueArray("symbol", hr24BTC));

┌ Info: Recompiling stale cache file /Users/drutjes/.julia/compiled/v1.0/Binance/polIw.ji for Binance [840d5de0-b432-11e8-2099-35be2bba8ecc]
└ @ Base loading.jl:1184


## Retrieve Data with interval 1 month "1M" and 1 minute "1m"

In [4]:
res = nothing;
# loop through all the symbols en retrieve kline data from binance api
current_datetime = now();
print("retrieving data from binance $current_datetime for :")

for symbol in symbols
    global res
    print(" $symbol")
    dfM = getBinanceKlineDataframe(symbol, now(UTC)-Month(6),now(); interval="1M")
    if(dfM == nothing)
        continue
    end

    # append most recent 5m data, if no trades occured skip this
    dfm = getBinanceKlineDataframe(symbol, now(UTC)-Minute(5),now(UTC); interval="1m")
    if(dfm == nothing)
        continue
    end

    append!(dfM,dfm)

    if res == nothing
        res = dfM;
    else
        append!(res,dfM)
    end
end

alldata = by(res, :symbol, df -> DataFrame(current = df[:close][end], min = minimum(df[:low]), changeMin = (df[:close][end] - minimum(df[:low]))/minimum(df[:low])*100, max =maximum(df[:high]), changeMax = (maximum(df[:high])-df[:close][end]) / maximum(df[:high])*100, trades =sum(df[:trades]),volume =sum(df[:volume])));


retrieving data from binance 2018-09-09T18:53:17.593 for : ADABTC ADXBTC AEBTC AGIBTC AIONBTC AMBBTC APPCBTC ARDRBTC ARKBTC ARNBTC ASTBTC BATBTC BCCBTC BCDBTC BCNBTC BCPTBTC BLZBTC BNBBTC BNTBTC BQXBTC BRDBTC BTGBTC BTSBTC CDTBTC CHATBTC CLOAKBTC CMTBTC CNDBTC CVCBTC DASHBTC DATABTC DENTBTC DGDBTC DLTBTC DNTBTC DOCKBTC EDOBTC ELFBTC ENGBTC ENJBTC EOSBTC ETCBTC ETHBTC EVXBTC FUELBTC FUNBTC GASBTC GNTBTC GRSBTC GTOBTC GVTBTC GXSBTC HCBTC HOTBTC HSRBTC ICNBTC ICXBTC INSBTC IOSTBTC IOTABTC IOTXBTC KEYBTC KMDBTC KNCBTC LENDBTC LINKBTC LOOMBTC LRCBTC LSKBTC LTCBTC LUNBTC MANABTC MCOBTC MDABTC MFTBTC MODBTC MTHBTC MTLBTC NANOBTC NASBTC NAVBTC NCASHBTC NEBLBTC NEOBTC NPXSBTC NULSBTC NXSBTC OAXBTC OMGBTC ONTBTC OSTBTC PHXBTC PIVXBTC POABTC POEBTC POLYBTC POWRBTC PPTBTC QKCBTC QLCBTC QSPBTC QTUMBTC RCNBTC RDNBTC REPBTC REQBTC RLCBTC RPXBTC SALTBTC SCBTC SKYBTC SNGLSBTC SNMBTC SNTBTC STEEMBTC STORJBTC STORMBTC STRATBTC SUBBTC SYSBTC THETABTC TNBBTC TNTBTC TRIGBTC TRXBTC TUSDBTC VENBTC VETBTC VIAB

## sort on volume/trades

In [5]:
sort!(alldata, [:volume,:trades]; rev=true);

## who is the loser ?

In [6]:
head(sort(alldata[1:40,:], [:changeMin]; rev=false))

,symbol,current,min,changeMin,max,changeMax,trades,volume
1,ADABTC,1.227e-5,1.201e-5,2.16486,4.19e-5,70.716,10013620,2.03832e10
2,FUNBTC,2.23e-6,2.17e-6,2.76498,6.51e-6,65.745,1659651,6.09706e9
3,SNGLSBTC,2.91e-6,2.83e-6,2.82686,2.295e-5,87.3203,1185198,2.31789e9
4,GTOBTC,1.002e-5,9.69e-6,3.40557,6.95e-5,85.5827,2833881,3.14644e9
5,LENDBTC,1.83e-6,1.76e-6,3.97727,1.086e-5,83.1492,1264670,3.55641e9
6,POEBTC,1.26e-6,1.2e-6,5.0,6.79e-6,81.4433,2420694,1.79134e10
